In [ ]:
# Run this cell to install DiffeRT and its dependencies, e.g., on Google Colab

try:
    import differt  # noqa: F401
except ImportError:
    import sys  # noqa: F401

    !{sys.executable} -m pip install differt[all]

# Coherence Map

In [2]:
import equinox as eqx
import jax.numpy as jnp
import numpy as np

from differt.geometry.paths import merge_groups
from differt.geometry.utils import path_lengths
from differt.plotting import draw_image, reuse, set_defaults
from differt.scene.sionna import download_sionna_scenes, get_sionna_scene
from differt.scene.triangle_scene import TriangleScene

In [3]:
download_sionna_scenes()  # Let's download Sionna scenes (from the main branch)

In [ ]:
set_defaults("plotly")

file = get_sionna_scene("simple_street_canyon")
scene = TriangleScene.load_xml(file)

scene = eqx.tree_at(lambda s: s.transmitters, scene, jnp.array([-33.0, 0.0, 32.0]))
batch = (50, 50)
scene_grid = scene.with_receivers_grid(*batch)
power = jnp.zeros(batch)
groups = jnp.zeros(batch, dtype=jnp.int32)

x, y, z = jnp.unstack(scene_grid.receivers, axis=-1)

with reuse() as fig:
    scene.plot()

    for order in range(3):
        for paths in scene_grid.compute_paths(order=order, chunk_size=1_000):
            new_groups = paths.mask_groups()
            groups = merge_groups(groups, new_groups)
            power += (paths.mask / path_lengths(paths.vertices) ** 2).sum(axis=-1)

    draw_image(
        np.asarray(power),
        x=np.asarray(x[0, :]),
        y=np.asarray(y[:, 0]),
        z0=float(z.ravel()[0]),
    )

fig

In [ ]:
with reuse("plotly") as fig:
    # We renumber unique indices to be between 0 and num_unique_groups (excluded)
    renumbered_groups = jnp.unique(groups, return_inverse=True)[1].reshape(groups.shape)
    scene.plot()
    draw_image(
        np.asarray(renumbered_groups),
        x=np.asarray(x[0, :]),
        y=np.asarray(y[:, 0]),
        z0=float(z.ravel()[0]),
    )

fig

In [ ]:
length_x = x.max() - x.min()
length_y = y.max() - y.min()
surface = length_x * length_y
surface_per_point = (
    surface / groups.size
)  # ~ Roughly, because RXs are not placed at centers of tiles

points_per_group = jnp.unique(groups, return_counts=True)[1]
points_per_group

In [ ]:
import matplotlib.pyplot as plt

surface_per_group = points_per_group * surface_per_point


plt.hist(
    surface_per_group[1:]
)  # TODO: find better way to remove group with not multipath components